In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,mean_absolute_error
from sklearn.model_selection import RepeatedStratifiedKFold
import matplotlib.pyplot as plt
from sklearn.tree import export_text
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
from pydot import graph_from_dot_data
import pandas as pd
import numpy as np
from numpy import nan
from graphviz import render
import seaborn as sns
from matplotlib import pyplot
import matplotlib.ticker as ticker
from chefboost import Chefboost as chef

In [45]:
!pip install chefboost

     |████████████████████████████████| 70 kB 7.1 MB/s eta 0:00:011


In [2]:
def min_max_scaling(df,columns):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm

In [3]:
columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","salary"]

In [4]:
adult_data = pd.read_csv('./data/adult.data',names=columns,index_col=False)

In [5]:
adult_test = pd.read_csv('./data/adult.test',names=columns,index_col=False,skiprows=1)

In [6]:
joined_data = pd.concat([adult_data,adult_test])

In [7]:
joined_data[columns] = joined_data[columns].replace([' ?','?','? '], nan)
adult_data[columns] = adult_data[columns].replace(' ?',nan)
adult_test[columns] = adult_test[columns].replace(' ?',nan)

In [8]:
joined_data['salary'] = joined_data['salary'].str.strip()
joined_data['salary'] = joined_data['salary'].str.replace(".","")

In [9]:
joined_data['salary'].unique()

array(['<=50K', '>50K'], dtype=object)

In [10]:
joined_data.isna().sum()

age                  0
workclass         2799
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
salary               0
dtype: int64

In [11]:
joined_data.dropna(inplace=True)
adult_data.dropna(inplace=True)

### KBinsDiscretizer

In [12]:
discretize_columns = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']

#### DISTRIBUTION OF DATA

In [ ]:
joined_data.hist()
pyplot.tight_layout()
pyplot.show()

In [13]:
data = joined_data[discretize_columns]

In [14]:
trans = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
data = trans.fit_transform(data)

In [15]:
dataset = pd.DataFrame(data,columns = discretize_columns)

In [16]:
joined_data.update(dataset)

In [ ]:
joined_data.head()

In [ ]:
dataset.hist()
pyplot.tight_layout()
pyplot.show()

uniform will preseve the probability distribution of each input

In [ ]:
enc = KBinsDiscretizer(n_bins=10, encode='ordinal',strategy='uniform')
X_binned = enc.fit_transform(np.expand_dims(joined_data['age'],axis=1))

In [ ]:
pyplot.hist(X_binned,bins=10)
pyplot.show()

In [ ]:
enc = KBinsDiscretizer(n_bins=4, encode='ordinal',strategy='uniform')
X_binned = enc.fit_transform(np.expand_dims(joined_data['fnlwgt'],axis=1))

In [ ]:
pyplot.hist(X_binned,bins=4)
pyplot.show()

### UPSAMPLING MINORITY CLASS

In [17]:
joined_data['salary'].value_counts()

<=50K    34014
>50K     11208
Name: salary, dtype: int64

In [18]:
df_majority = joined_data[joined_data.salary=='<=50K']
df_minority = joined_data[joined_data.salary=='>50K']

In [19]:
len(df_majority)

34014

In [20]:
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=123)

In [21]:
joined_data = pd.concat([df_majority, df_minority_upsampled])

In [22]:
joined_data.salary.value_counts()

>50K     34014
<=50K    34014
Name: salary, dtype: int64

### DOWN SAMPLING MAJORITY CLASS

In [ ]:
joined_data['salary'].value_counts()

In [ ]:
df_majority = joined_data[joined_data.salary=='<=50K']
df_minority = joined_data[joined_data.salary=='>50K']

In [ ]:
len(df_majority)

In [ ]:
df_majority_downsampled = resample(df_majority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_minority),    # to match majority class
                                 random_state=123)

In [ ]:
joined_data = pd.concat([df_minority, df_majority_downsampled])

In [ ]:
joined_data.salary.value_counts()

In [23]:
numerical_columns = joined_data.select_dtypes(include='int64').columns.tolist()
numerical_df = joined_data.select_dtypes(include='int64')
categorical_columns = joined_data.select_dtypes(include='object').columns.tolist()
categorical_columns.remove('salary')

### ONE HOT ENCODING

In [ ]:
joined_data = pd.get_dummies(joined_data,columns=categorical_columns)

In [ ]:
joined_data.head()

In [ ]:
joined_data = pd.get_dummies(joined_data, columns=['salary'],drop_first=True)

In [ ]:
joined_data.dtypes

### NORMALIZE DF

In [ ]:
joined_data = min_max_scaling(joined_data,numerical_columns)

In [ ]:
joined_data.dtypes

### LABEL ENCODING

In [24]:
joined_data['education'] = joined_data['education'].astype('category')
joined_data['workclass'] = joined_data['workclass'].astype('category')
joined_data['marital-status'] = joined_data['marital-status'].astype('category')
joined_data['occupation'] = joined_data['occupation'].astype('category')
joined_data['relationship'] = joined_data['relationship'].astype('category')
joined_data['race'] = joined_data['race'].astype('category')
joined_data['sex'] = joined_data['sex'].astype('category')
joined_data['native-country'] = joined_data['native-country'].astype('category')
joined_data['salary'] = joined_data['salary'].astype('category')

In [25]:
joined_data['education_cat'] = joined_data['education'].cat.codes
joined_data['workclass_cat'] = joined_data['workclass'].cat.codes
joined_data['ms_cat'] = joined_data['marital-status'].cat.codes
joined_data['occupation_cat'] = joined_data['occupation'].cat.codes
joined_data['relationship_cat'] = joined_data['relationship'].cat.codes
joined_data['race_cat'] = joined_data['race'].cat.codes
joined_data['sex_cat'] = joined_data['sex'].cat.codes
joined_data['native-country_cat'] = joined_data['native-country'].cat.codes
joined_data['salary_cat'] = joined_data['salary'].cat.codes

In [26]:
joined_data['salary'].unique()

['<=50K', '>50K']
Categories (2, object): ['<=50K', '>50K']

In [27]:
for column in joined_data.select_dtypes(include='float64').columns.tolist():
    joined_data[column] = joined_data[column].astype('float32')

In [28]:
columns = joined_data.select_dtypes(include=['float32','int8']).columns.tolist()

In [29]:
columns.remove('salary_cat')

In [30]:
#X = adult_data[['education_cat','workclass_cat','ms_cat','occupation_cat','relationship_cat','sex_cat','race_cat','native-country_cat']]
X = joined_data[columns]
#X = joined_data.loc[:,joined_data.columns != 'salary_>50K']

In [31]:
y = joined_data['salary_cat']

### PCA

In [ ]:
pca = PCA(n_components =3)
principalComponents = pca.fit_transform(X)

In [ ]:
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['pc1','pc2','pc3'])

In [ ]:
#principalDf["Cluster"] = y.tolist()

In [ ]:
principalDf.head()

In [ ]:
df = pd.DataFrame({'var':pca.explained_variance_ratio_,
             'PC':['PC1','PC2','PC3']})

In [ ]:
ax = plt.axes()
sns.barplot(x='PC',y="var", 
           data=df, color="c")
ax.set_title('Principle Components')
plt.show()

In [ ]:
sns.lmplot( x="pc1", y="pc2",
  data=principalDf, 
  fit_reg=False, 
  hue='Cluster', # color by cluster
  legend=True)

In [ ]:
X = principalDf

In [32]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### FEATURE IMPORTANCE USING RANDOM FOREST

In [ ]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

In [ ]:
clf.feature_importances_

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=X_train.columns.tolist()).sort_values(ascending=False)
feature_impfeature_imp = pd.Series(clf.feature_importances_,index=X_train.columns.tolist()).sort_values(ascending=False)
feature_imp

In [ ]:
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Important Features")
plt.legend()
plt.show()

### FEATURE IMPORTANCE USING OTHER METHODS

In [ ]:
test = SelectKBest(score_func=f_classif, k=4)

In [ ]:
fit = test.fit(X, y)

In [ ]:
set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)

In [ ]:
print(features[0:5,:])

In [ ]:
model = ExtraTreesClassifier(n_estimators=len(columns))
model.fit(X, y)
feature_scores = model.feature_importances_.tolist()
feature_scores.sort(reverse=True)

In [ ]:
# for index,feature in enumerate(feature_scores):
#     print('{}:{}'.format(columns[index], feature))

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
variables = columns
importance = feature_scores
ax.bar(variables,importance)
plt.xticks(rotation=45)
plt.tight_layout()
plt.title('feature importance')
plt.show()

In [ ]:
drop_columns=['education_cat','workclass_cat','ms_cat','occupation_cat','relationship_cat','race_cat','sex_cat','native-country_cat']

In [ ]:
X.drop(drop_columns,axis=1)

### OUTLIER DETECTION

In [ ]:
iso = IsolationForest(contamination=0.1)

In [ ]:
yhat = iso.fit_predict(X_train)

In [ ]:
mask = yhat != -1

In [ ]:
X_train, y_train = X_train[mask], y_train[mask]

In [ ]:
len(X_train)

### DECISION TREE CLASSIFICATION

In [34]:
model = DecisionTreeClassifier(random_state=0,criterion='gini',ccp_alpha = 0.010)

In [ ]:
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

In [ ]:
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
path = model.cost_complexity_pruning_path(X_train, y_train)

In [ ]:
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
len(ccp_alphas)

In [35]:
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.01, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [36]:
#classes =list(map(lambda x:'>50K' if x else '>=50k',y_train.tolist()))

In [44]:
tree_rules = export_text(model, feature_names=list(X_train.columns))

In [43]:
print(tree_rules)

|--- relationship_cat <= 0.50
|   |--- education_cat <= 6.50
|   |   |--- weights: [1382.00, 623.00] class: 0
|   |--- education_cat >  6.50
|   |   |--- weights: [6271.00, 18866.00] class: 1
|--- relationship_cat >  0.50
|   |--- relationship_cat <= 4.50
|   |   |--- weights: [16887.00, 3874.00] class: 0
|   |--- relationship_cat >  4.50
|   |   |--- weights: [811.00, 2307.00] class: 1



In [ ]:
n_nodes = model.tree_.node_count
n_nodes

### PRUNING TREE

In [ ]:
model.cost_complexity_pruning_path(X_train, y_train)

In [ ]:
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]
fig, ax = plt.subplots(2, 1)
ax[0].plot(ccp_alphas, node_counts, marker='o', drawstyle="steps-post")
ax[0].set_xlabel("alpha")
ax[0].set_ylabel("number of nodes")
ax[0].set_title("Number of nodes vs alpha")
ax[1].plot(ccp_alphas, depth, marker='o', drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

In [ ]:
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

### MAX DEPTH

In [ ]:
max_depth = []
acc_gini_train = []
acc_gini_test = []

acc_entropy = []
for i in range(1,50):
    dtree = DecisionTreeClassifier(criterion='gini', max_depth=i)
    dtree.fit(X_train, y_train)
    pred = dtree.predict(X_test)
    acc_gini_test.append(accuracy_score(y_test, pred))
    
    
    pred = dtree.predict(X_train)
    acc_gini_train.append(accuracy_score(y_train, pred))
    
    
    ####
    dtree = DecisionTreeClassifier(criterion='entropy', max_depth=i)
    dtree.fit(X_train, y_train)
    pred = dtree.predict(X_test)
    acc_entropy.append(accuracy_score(y_test, pred))
    ####
    max_depth.append(i)
d = pd.DataFrame({'acc_gini_train':pd.Series(acc_gini_train),
 'acc_gini_test':pd.Series(acc_gini_test),
 'acc_entropy':pd.Series(acc_entropy),
 'max_depth':pd.Series(max_depth)})
# visualizing changes in parameters
plt.plot('max_depth','acc_gini_train', data=d, label='gini_train')
plt.plot('max_depth','acc_gini_test', data=d, label='gini_test')
plt.plot('max_depth','acc_entropy', data=d, label='entropy')
plt.xlabel('max_depth')
# plt.xticks(max_depth)
plt.axvline(x=5, color='r')
ax = plt.axes()
ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
plt.ylabel('accuracy')
plt.legend()

### VISUALISE DECISION TREE

In [ ]:
dot_data = StringIO()

In [ ]:
classes =list(map(lambda x:'>50K' if x else '>=50k',y_train.tolist()))

In [ ]:
export_graphviz(model, out_file=dot_data, feature_names=X.columns,class_names=classes,rounded = True, 
                precision = 2, filled = True)

In [ ]:
(graph,) = graph_from_dot_data(dot_data.getvalue())

In [ ]:
Image(graph.create_png())

### PREDICTIONS

In [ ]:
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
# salary = np.array(y_test).argmax(axis=1)
# predictions = np.array(y_pred).argmax(axis=1)
cm = confusion_matrix(y_test, y_pred.tolist())

In [ ]:
ax = plt.axes()
ax.set_title('normalized confussion matrix')
sns.heatmap(cm/np.sum(cm), annot=True, 
            fmt='.2%', cmap='Blues',ax = ax)
plt.show()

In [ ]:
print("TRAIN_Accuracy:",accuracy_score(y_train, y_pred_train.tolist()))
print("TRAIN_MSE:",mean_absolute_error(y_train, y_pred_train.tolist()))

print("TEST_Accuracy:",accuracy_score(y_test, y_pred.tolist()))
print("TEST_MSE:",mean_absolute_error(y_test, y_pred.tolist()))